In [3]:
from keras import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from keras import optimizers
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# create the train and validate directories
train_dir = 'data/train'
val_dir = 'data/test'
# generate a dataset from image files in our directory from scale 1 to 255
# because we defined our images in pixels (just an array consisting of values from 1 to 255)
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
# create our generator training and validation
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),  # 48 x 48 pixel images
    batch_size=64,  # number of training examples uterlized in one iteration
    color_mode='grayscale',  # for grayscale and white and black images
    class_mode='categorical')  # because we have catergories of faces (angry, sad, etc)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(48, 48),
    batch_size=64,
    color_mode='grayscale',
    class_mode='categorical')

# create our model for training
# initializing our sequential model
# Sequential api: allows you to create model layer by layer
# but does not allow sharing layers with multiple inputs and outputs
# Functional api: allows you to create models with alot of flexibility that you can easily
# connect layer to more than just the previous next layer (you can connect layers to literally any layer)
model = Sequential()
# Convolution filter to an input
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=optimizers.adam_v2.Adam(learning_rate=0.0001, decay=1e-6),
              metrics=['accuracy'])

model_info = model.fit(train_generator, steps_per_epoch=28709 // 64, epochs=50, validation_data=validation_generator, validation_steps=7178 // 64)

# Save trained model
model.save('hdp_keras.h5')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Epoch 1/50
448/448 [==============================] - 27s 58ms/step - loss: 1.7992 - accuracy: 0.2602 - val_loss: 1.7036 - val_accuracy: 0.3419
Epoch 2/50
448/448 [==============================] - 17s 37ms/step - loss: 1.6268 - accuracy: 0.3677 - val_loss: 1.5323 - val_accuracy: 0.4212
Epoch 3/50
448/448 [==============================] - 16s 36ms/step - loss: 1.5203 - accuracy: 0.4120 - val_loss: 1.4550 - val_accuracy: 0.4463
Epoch 4/50
448/448 [==============================] - 16s 37ms/step - loss: 1.4466 - accuracy: 0.4452 - val_loss: 1.3766 - val_accuracy: 0.4780
Epoch 5/50
448/448 [==============================] - 17s 37ms/step - loss: 1.3856 - accuracy: 0.4732 - val_loss: 1.3343 - val_accuracy: 0.4967
Epoch 6/50
448/448 [==============================] - 16s 37ms/step - loss: 1.3325 - accuracy: 0.4933 - val_loss: 1.2867 - val_accuracy: 0.5145
Epoch 7/50
448/448 [==============================]